<a href="https://colab.research.google.com/github/Umesh-chandra-2006/Job-Fit-Analyzer/blob/main/IBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task
Build a comprehensive AI-powered Job Application Assistant in Google Colab. This assistant should be able to:
1.  **Analyze job fit**: Evaluate a resume against a job description, providing a match score, identifying matching and missing skills, suggesting keyword recommendations, generating a cover letter introduction, proposing interview questions, and offering actionable improvements.
2.  **Compare multiple jobs**: Analyze a single resume against several job descriptions and rank them by match score.
3.  **Provide general resume improvements**: Offer generic suggestions to enhance a resume.
4.  **Implement robust features**: This includes API configuration for Google Gemini, text preprocessing, structured AI analysis prompts, user-friendly output formatting, interactive and demo modes, and the ability to save analysis results.
The solution should include all necessary installations, configurations, functions, and a main execution block to demonstrate its full functionality.

## Setup and Installation

### Subtask:
Install the necessary Python libraries including `google-generativeai`, `re`, and `json`. Verify the installation by printing a success message.


**Reasoning**:
To fulfill the subtask, the first step is to install the `google-generativeai` library. The other specified libraries (`re` and `json`) are built-in and do not require explicit installation. After installation, a success message will be printed to confirm the step is completed.



In [ ]:
!pip install google-generativeai

print("Required libraries (google-generativeai, re, and json) have been successfully handled.")

**Reasoning**:
The previous code block failed because `pip install` was used directly in Python syntax, which is incorrect. In a Colab environment, shell commands like `pip install` need to be prefixed with an exclamation mark `!` to be executed correctly.



In [ ]:
get_ipython().system('pip install google-generativeai')

print("Required libraries (google-generativeai, re, and json) have been successfully handled.")

## API Configuration

### Subtask:
Configure the Google Gemini API. This involves setting a placeholder for the `API_KEY` (which the user must replace), initializing the GenerativeModel, and testing the API connection to ensure it's working correctly. Include error handling for API configuration issues.


#### **Important: Setup your Google API Key**

1.  Go to [Google AI Studio](https://aistudio.google.com/app/apikey) and create an API key.
2.  In Google Colab, click on the "🔑 Secrets" icon on the left panel.
3.  Add a new secret with the name `GOOGLE_API_KEY` and paste your API key as the value.
4.  Ensure that "Notebook access" is enabled for `GOOGLE_API_KEY`.

**Reasoning**:
Now that the user has been instructed on how to set up the API key, I will provide the Python code to configure the Gemini API, initialize the model, and test the connection, including error handling as specified in the subtask instructions.



In [ ]:
import google.generativeai as gemini
import os
from google.colab import userdata

try:
    # Get the API key from Colab's secrets using userdata.get()
    api_key = userdata.get("GOOGLE_API_KEY")

    if not api_key:
        raise ValueError("GOOGLE_API_KEY not found in Colab secrets. Please set it up as instructed.")

    # Configure the API
    gemini.configure(api_key=api_key)

    # Initialize the GenerativeModel globally so it can be used by other functions
    model = gemini.GenerativeModel('gemini-2.5-flash')

    # Test the API connection
    _ = model.generate_content('Hello')

    print("Google Gemini API configured and tested successfully!")
except ValueError as e:
    print(f"API Configuration Error: {e}")
    print("Please ensure your GOOGLE_API_KEY is correctly set in Colab secrets and Notebook access is enabled.")
except Exception as e:
    print(f"An unexpected error occurred during API configuration or testing: {e}")
    print("Please check your internet connection and API key validity.")

## Text Preprocessing Functions

### Subtask:
Implement the `clean_text`, `validate_input`, and `extract_keywords` functions. `clean_text` should remove extra whitespace and special characters, `validate_input` should check for minimum length, and `extract_keywords` should identify potential skill keywords from text.


**Reasoning**:
To implement the subtask, I will define the `clean_text`, `validate_input`, and `extract_keywords` functions as specified, including importing the `re` and `json` modules.



In [ ]:
import re
import json

def clean_text(text: str) -> str:
    """
    Cleans the input text by converting to lowercase, removing non-alphanumeric
    characters (keeping spaces), replacing multiple spaces with a single space,
    and stripping leading/trailing whitespace.
    """
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)  # Keep only alphanumeric and spaces
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with single space
    return text.strip()

def validate_input(text: str, min_length: int = 50) -> bool:
    """
    Validates if the input text is not None and meets a minimum length requirement.
    """
    return text is not None and len(str(text).strip()) >= min_length

def extract_keywords(text: str) -> list[str]:
    """
    Extracts potential skill keywords from text. A keyword is defined as a word
    consisting of at least two alphabetic characters.
    """
    if not isinstance(text, str):
        return []
    # Find words consisting of at least two alphabetic characters
    keywords = re.findall(r'\b[a-z]{2,}\b', text.lower())
    return sorted(list(set(keywords)))

## Core AI Analysis Functions

### Subtask:
Develop the `build_analysis_prompt` function to construct a detailed prompt for the AI analysis, ensuring it matches the specified JSON output format. Then, implement the `analyze_job_fit` function to orchestrate the cleaning, validation, prompt building, Gemini API call, and JSON response parsing. Include robust error handling for input validation and JSON decoding.


**Reasoning**:
I will define the `build_analysis_prompt` function to construct the AI prompt with the specified JSON output format and then implement the `analyze_job_fit` function, incorporating text cleaning, input validation, AI model interaction, and robust error handling for API calls and JSON parsing, as per the subtask requirements.



In [ ]:
import json

def build_analysis_prompt(resume_text: str, job_description_text: str) -> str:
    """
    Constructs a detailed prompt for the AI to analyze job fit.
    The prompt includes instructions for evaluation and the required JSON output format.
    """
    # Define the desired JSON output format
    model_output_format = json.dumps({
        "match_score": "<percentage e.g., 75%>",
        "matching_skills": ["<skill1>", "<skill2>"],
        "missing_skills": ["<skill1>", "<skill2>"],
        "keyword_suggestions": ["<keyword1>", "<keyword2>"],
        "cover_letter_intro": "<a compelling cover letter introduction based on fit>",
        "interview_questions": ["<question1>", "<question2>"],
        "actionable_improvements": ["<improvement1>", "<improvement2>"]
    }, indent=2)

    prompt = f"""
    You are an expert HR assistant. Your task is to meticulously analyze a given resume against a job description.
    Provide a comprehensive evaluation that includes a match score, identified skills, suggestions, and actionable improvements.

    Here is the RESUME:
    {resume_text}

    Here is the JOB DESCRIPTION:
    {job_description_text}

    Analyze the resume against the job description and generate a response in a JSON format.
    The JSON object should strictly adhere to the following structure:
    {model_output_format}

    Ensure all arrays contain relevant items and the cover_letter_intro is compelling.
    The match_score should reflect the overall fit of the resume to the job description.
    The `keyword_suggestions` should be based on keywords present in the job description but not explicitly mentioned or highlighted in the resume.
    `actionable_improvements` should be specific and constructive suggestions for the candidate to improve their resume for this job or similar roles.
    """
    return prompt

def analyze_job_fit(resume_text: str, job_description_text: str) -> dict:
    """
    Analyzes the fit between a resume and a job description using the Google Gemini API.
    Includes validation, text cleaning, prompt generation, API call, and JSON parsing with error handling.
    """
    if not validate_input(resume_text):
        raise ValueError("Resume text is too short or invalid. Minimum length required.")
    if not validate_input(job_description_text):
        raise ValueError("Job description text is too short or invalid. Minimum length required.")

    cleaned_resume = clean_text(resume_text)
    cleaned_job_description = clean_text(job_description_text)

    prompt = build_analysis_prompt(cleaned_resume, cleaned_job_description)

    try:
        # Ensure 'model' is initialized from the API configuration step
        if 'model' not in globals():
            raise RuntimeError("Gemini model not initialized. Please run API configuration step.")

        response = model.generate_content(prompt)
        # Assuming the model returns a JSON string directly in response.text
        ai_response_text = response.text

        # Remove markdown code block fences if present
        ai_response_text = ai_response_text.replace('```json', '').replace('```', '').strip()

        # Attempt to parse the JSON response
        try:
            analysis_results = json.loads(ai_response_text)
            return analysis_results
        except json.JSONDecodeError as e:
            print(f"JSON parsing error: {e}")
            print(f"AI Raw Response: {ai_response_text}")
            raise ValueError("AI response is not valid JSON.")

    except Exception as e:
        print(f"API call error: {e}")
        raise RuntimeError(f"Failed to get AI analysis: {e}")

print("Functions `build_analysis_prompt` and `analyze_job_fit` defined successfully.")

**Reasoning**:
The previous code block failed due to a `SyntaxError` related to incorrectly nesting triple quotes within an f-string. I need to correct the way `resume_text` and `job_description_text` are embedded in the multi-line f-string.



In [ ]:
import json

def build_analysis_prompt(resume_text: str, job_description_text: str) -> str:
    """
    Constructs a detailed prompt for the AI to analyze job fit.
    The prompt includes instructions for evaluation and the required JSON output format.
    """
    # Define the desired JSON output format
    model_output_format = json.dumps({
        "match_score": "<percentage e.g., 75%>",
        "matching_skills": ["<skill1>", "<skill2>"],
        "missing_skills": ["<skill1>", "<skill2>"],
        "keyword_suggestions": ["<keyword1>", "<keyword2>"],
        "cover_letter_intro": "<a compelling cover letter introduction based on fit>",
        "interview_questions": ["<question1>", "<question2>"],
        "actionable_improvements": ["<improvement1>", "<improvement2>"]
    }, indent=2)

    prompt = f"""
    You are an expert HR assistant. Your task is to meticulously analyze a given resume against a job description.
    Provide a comprehensive evaluation that includes a match score, identified skills, suggestions, and actionable improvements.

    Here is the RESUME:
    {resume_text}

    Here is the JOB DESCRIPTION:
    {job_description_text}

    Analyze the resume against the job description and generate a response in a JSON format.
    The JSON object should strictly adhere to the following structure:
    {model_output_format}

    Ensure all arrays contain relevant items and the cover_letter_intro is compelling.
    The match_score should reflect the overall fit of the resume to the job description.
    The `keyword_suggestions` should be based on keywords present in the job description but not explicitly mentioned or highlighted in the resume.
    `actionable_improvements` should be specific and constructive suggestions for the candidate to improve their resume for this job or similar roles.
    """
    return prompt

def analyze_job_fit(resume_text: str, job_description_text: str) -> dict:
    """
    Analyzes the fit between a resume and a job description using the Google Gemini API.
    Includes validation, text cleaning, prompt generation, API call, and JSON parsing with error handling.
    """
    if not validate_input(resume_text):
        raise ValueError("Resume text is too short or invalid. Minimum length required.")
    if not validate_input(job_description_text):
        raise ValueError("Job description text is too short or invalid. Minimum length required.")

    cleaned_resume = clean_text(resume_text)
    cleaned_job_description = clean_text(job_description_text)

    prompt = build_analysis_prompt(cleaned_resume, cleaned_job_description)

    try:
        # Ensure 'model' is initialized from the API configuration step
        if 'model' not in globals():
            raise RuntimeError("Gemini model not initialized. Please run API configuration step.")

        response = model.generate_content(prompt)
        # Assuming the model returns a JSON string directly in response.text
        ai_response_text = response.text

        # Remove markdown code block fences if present
        ai_response_text = ai_response_text.replace('```json', '').replace('```', '').strip()

        # Attempt to parse the JSON response
        try:
            analysis_results = json.loads(ai_response_text)
            return analysis_results
        except json.JSONDecodeError as e:
            print(f"JSON parsing error: {e}")
            print(f"AI Raw Response: {ai_response_text}")
            raise ValueError("AI response is not valid JSON.")

    except Exception as e:
        print(f"API call error: {e}")
        raise RuntimeError(f"Failed to get AI analysis: {e}")

print("Functions `build_analysis_prompt` and `analyze_job_fit` defined successfully.")

## Output Formatting Functions

### Subtask:
Create `format_match_score` to color-code and assess the match percentage. Implement `display_analysis` to present the AI's analysis results in a user-friendly, formatted manner, including match score, matching/missing skills, keyword recommendations, cover letter intro, interview questions, and actionable improvements. Also, implement `save_analysis` to save the results to a text file.


**Reasoning**:
I will implement the `format_match_score`, `display_analysis`, and `save_analysis` functions as specified, including logic for color-coding the score, formatting the AI analysis for display, and writing the results to a text file.



In [ ]:
import os

def format_match_score(score_str: str) -> str:
    """
    Formats the match score with color-coding and a textual assessment.
    Assumes score_str is like '75%'.
    """
    try:
        score = int(score_str.replace('%', '').strip())
        if score >= 80:
            color = "\033[92m" # Green
            assessment = "Excellent Fit"
        elif score >= 60:
            color = "\033[93m" # Yellow
            assessment = "Good Fit"
        else:
            color = "\033[91m" # Red
            assessment = "Moderate/Low Fit"
        return f"{color}{score_str} - {assessment}\033[0m"
    except (ValueError, AttributeError):
        return f"N/A - Invalid Score: {score_str}"

def display_analysis(analysis_results: dict):
    """
    Displays the AI's analysis results in a user-friendly, formatted manner.
    """
    print("\n" + "="*50)
    print("           JOB APPLICATION ANALYSIS           ")
    print("="*50 + "\n")

    match_score = analysis_results.get('match_score', 'N/A')
    print(f"Match Score: {format_match_score(match_score)}\n")

    print("\033[1mMatching Skills:\033[0m")
    for skill in analysis_results.get('matching_skills', []):
        print(f"- {skill}")
    print("\n")

    print("\033[1mMissing Skills:\033[0m")
    for skill in analysis_results.get('missing_skills', []):
        print(f"- {skill}")
    print("\n")

    print("\033[1mKeyword Suggestions:\033[0m")
    for keyword in analysis_results.get('keyword_suggestions', []):
        print(f"- {keyword}")
    print("\n")

    print("\033[1mCover Letter Introduction:\033[0m")
    print(analysis_results.get('cover_letter_intro', 'N/A'))
    print("\n")

    print("\033[1mPotential Interview Questions:\033[0m")
    for i, question in enumerate(analysis_results.get('interview_questions', [])):
        print(f"{i+1}. {question}")
    print("\n")

    print("\033[1mActionable Improvements:\033[0m")
    for i, improvement in enumerate(analysis_results.get('actionable_improvements', [])):
        print(f"{i+1}. {improvement}")
    print("\n" + "="*50 + "\n")

def save_analysis(analysis_results: dict, resume_text: str, job_description_text: str, filename: str = "job_analysis_report.txt"):
    """
    Saves the complete analysis, resume, and job description to a text file.
    """
    try:
        with open(filename, 'w', encoding='utf-8') as f:
            f.write("JOB APPLICATION ANALYSIS REPORT\n")
            f.write("="*50 + "\n\n")

            f.write("Resume:\n")
            f.write("-"*50 + "\n")
            f.write(resume_text + "\n\n")

            f.write("Job Description:\n")
            f.write("-"*50 + "\n")
            f.write(job_description_text + "\n\n")

            f.write("AI Analysis Results:\n")
            f.write("-"*50 + "\n")

            match_score = analysis_results.get('match_score', 'N/A')
            f.write(f"Match Score: {match_score}\n\n")

            f.write("Matching Skills:\n")
            for skill in analysis_results.get('matching_skills', []):
                f.write(f"- {skill}\n")
            f.write("\n")

            f.write("Missing Skills:\n")
            for skill in analysis_results.get('missing_skills', []):
                f.write(f"- {skill}\n")
            f.write("\n")

            f.write("Keyword Suggestions:\n")
            for keyword in analysis_results.get('keyword_suggestions', []):
                f.write(f"- {keyword}\n")
            f.write("\n")

            f.write("Cover Letter Introduction:\n")
            f.write(analysis_results.get('cover_letter_intro', 'N/A') + "\n\n")

            f.write("Potential Interview Questions:\n")
            for i, question in enumerate(analysis_results.get('interview_questions', [])):
                f.write(f"{i+1}. {question}\n")
            f.write("\n")

            f.write("Actionable Improvements:\n")
            for i, improvement in enumerate(analysis_results.get('actionable_improvements', [])):
                f.write(f"{i+1}. {improvement}\n")
            f.write("\n")

            f.write("="*50 + "\n")
        print(f"Analysis successfully saved to '{filename}'")
    except IOError as e:
        print(f"Error saving analysis to file: {e}")

print("Functions `format_match_score`, `display_analysis`, and `save_analysis` defined successfully.")

## Interactive User Interface

### Subtask:
Develop `run_interactive_analysis` to guide the user through pasting their resume and job description, then initiating the analysis and displaying the results. Also, implement `run_demo_analysis` with predefined sample resume and job description texts for quick testing and demonstration purposes.


**Reasoning**:
I will implement the `run_interactive_analysis` and `run_demo_analysis` functions as instructed, incorporating user input, input validation, API calls, and result display and saving, to fulfill the subtask.



In [ ]:
def run_interactive_analysis():
    """
    Guides the user through an interactive job fit analysis.
    """
    print("\n--- Interactive Job Fit Analysis ---")
    print("Please provide your resume and the job description.")

    resume_text = ""
    while not validate_input(resume_text):
        print("\nEnter your RESUME text (minimum 50 characters):")
        resume_text = input().strip()
        if not validate_input(resume_text):
            print("Resume text is too short. Please provide more details.")

    job_description_text = ""
    while not validate_input(job_description_text):
        print("\nEnter the JOB DESCRIPTION text (minimum 50 characters):")
        job_description_text = input().strip()
        if not validate_input(job_description_text):
            print("Job description text is too short. Please provide more details.")

    print("\nAnalyzing your resume against the job description... Please wait.")
    try:
        analysis_results = analyze_job_fit(resume_text, job_description_text)
        display_analysis(analysis_results)
        save_analysis(analysis_results, resume_text, job_description_text, "interactive_analysis_report.txt")
    except (ValueError, RuntimeError) as e:
        print(f"Error during analysis: {e}")
        print("Please review your input and API configuration.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

def run_demo_analysis():
    """
    Runs a job fit analysis with pre-defined sample resume and job description texts.
    """
    print("\n--- Running Demo Job Fit Analysis ---")

    sample_resume = """
    John Doe
    john.doe@email.com | (123) 456-7890 | LinkedIn: linkedin.com/in/johndoe

    Summary
    Highly motivated software engineer with 5 years of experience in Python, Java, and cloud platforms (AWS, Azure). Proven ability to develop and deploy scalable web applications and data processing pipelines.

    Skills
    Programming: Python, Java, JavaScript, C++
    Web Technologies: Django, Flask, React, Node.js
    Databases: PostgreSQL, MongoDB, SQL
    Cloud Platforms: AWS, Azure, Docker, Kubernetes
    Tools: Git, Jira, Agile, Scrum
    Data Science: Pandas, NumPy, Scikit-learn, Machine Learning

    Experience
    Senior Software Engineer | Tech Solutions Inc. | New York
    Jan 2021 - Present
    - Led the development of a microservices architecture using Python and Flask, reducing latency by 20%.
    - Designed and implemented data pipelines for large-scale data processing on AWS.
    - Mentored junior engineers and conducted code reviews, improving code quality.

    Software Engineer | Innovate Corp | San Francisco
    Aug 2018 - Dec 2020
    - Developed and maintained RESTful APIs using Java and Spring Boot.
    - Collaborated with cross-functional teams to define, design, and ship new features.
    - Optimized database queries, resulting in a 15% performance improvement.

    Education
    Master of Science in Computer Science | Stanford University | Sep 2017 - May 2018
    Bachelor of Science in Computer Engineering | University of California, Berkeley | Sep 2013 - May 2017
    """

    sample_job_description = """
    Job Title: Senior Software Engineer
    Location: Remote

    About Us:
    We are seeking a highly skilled Senior Software Engineer to join our dynamic team. You will be responsible for designing, developing, and maintaining scalable software solutions.

    Responsibilities:
    - Develop and implement high-quality software using Python and Java.
    - Work with cloud platforms like AWS and Kubernetes.
    - Participate in code reviews and mentor junior team members.
    - Contribute to architectural decisions and system design.

    Requirements:
    - Bachelor's degree in Computer Science or a related field.
    - 5+ years of experience in software development.
    - Strong proficiency in Python, Java, and related frameworks (e.g., Django, Spring).
    - Experience with cloud platforms (AWS, Azure) and containerization (Docker, Kubernetes).
    - Excellent problem-solving and communication skills.
    - Experience with machine learning principles is a plus.
    """

    print("\nUsing pre-defined sample texts for analysis. Please wait.")
    try:
        analysis_results = analyze_job_fit(sample_resume, sample_job_description)
        display_analysis(analysis_results)
        save_analysis(analysis_results, sample_resume, sample_job_description, "demo_analysis_report.txt")
    except (ValueError, RuntimeError) as e:
        print(f"Error during demo analysis: {e}")
        print("Please ensure your API is configured correctly and try again.")
    except Exception as e:
        print(f"An unexpected error occurred during demo analysis: {e}")

print("Functions `run_interactive_analysis` and `run_demo_analysis` defined successfully.")

## Main Execution Block

### Subtask:
Create a main execution block that provides options to the user to either run the `run_demo_analysis()` or `run_interactive_analysis()`, as described in the project specification.


**Reasoning**:
I will define the `main` function with a loop to present analysis options (demo, interactive, exit) to the user, handle their input, call the corresponding functions, and manage invalid choices, then add the `if __name__ == '__main__':` block to execute it.



In [122]:
def main():
    """
    Main execution block to provide options for running demo, interactive, or general resume analysis.
    """
    print("\nWelcome to the AI-powered Job Application Assistant!")

    while True:
        print("\n--- Choose an Option ---")
        print("1. Run Demo Analysis")
        print("2. Run Interactive Analysis")
        print("3. Get General Resume Improvements")
        print("4. Exit")
        choice = input("Enter your choice (1, 2, 3, or 4): ").strip()

        if choice == '1':
            run_demo_analysis()
        elif choice == '2':
            run_interactive_analysis()
        elif choice == '3':
            print("\n--- General Resume Improvements ---")
            resume_text = ""
            while not validate_input(resume_text, min_length=100): # Higher min_length for general advice
                print("\nEnter your RESUME text for general improvements (minimum 100 characters):")
                resume_text = input().strip()
                if not validate_input(resume_text, min_length=100):
                    print("Resume text is too short. Please provide more details.")

            print("\nAnalyzing your resume for general improvements... Please wait.")
            try:
                improvements = generate_resume_improvements(resume_text)
                display_resume_improvements(improvements)
            except (ValueError, RuntimeError) as e:
                print(f"Error generating resume improvements: {e}")
                print("Please review your input and API configuration.")
            except Exception as e:
                print(f"An unexpected error occurred during resume improvement generation: {e}")
        elif choice == '4':
            print("Exiting the Job Application Assistant. Goodbye!")
            break
        else:
            print("Invalid choice. Please enter 1, 2, 3, or 4.")

In [ ]:
import os

# List all files and directories in the current working directory
print("Files in the current directory:")
for filename in os.listdir('/content/'):
    print(f"- {filename}")

## Usage Examples and Testing Checklist

### Subtask:
Add code examples demonstrating how to use the implemented functions, including basic analysis, interactive mode, multiple job comparison, and general resume review. Incorporate the testing checklist and presentation guide into the notebook documentation as markdown cells.


### Usage Examples

To use the Job Application Assistant, you have a few options:

1.  **Run Demo Analysis:** This option uses pre-defined resume and job description texts to quickly demonstrate the assistant's capabilities without requiring user input. It's useful for a quick overview.
    *   Select option `1` when prompted in the main menu.

2.  **Run Interactive Analysis:** This option allows you to paste your own resume and a job description for a personalized analysis. The assistant will then evaluate the fit and provide detailed feedback.
    *   Select option `2` when prompted in the main menu.
    *   You will be asked to paste your resume text and then the job description text.

3.  **General Resume Improvements:** (Requires manual call for now, will be integrated into main menu later)
    *   You can call `generate_resume_improvements(your_resume_text)` with your resume.
    *   Then, `display_resume_improvements(suggestions)` to see general suggestions.

To begin, execute the final code cell that calls `main()`.

### Testing Checklist

To ensure all functionalities are working as expected, please verify the following:

1.  **Basic Job Fit Analysis (Demo Mode):**
    *   Run the demo analysis (Option 1 in main menu).
    *   Verify that the output includes a match score, matching skills, missing skills, keyword suggestions, a cover letter intro, interview questions, and actionable improvements.
    *   Check that the output is well-formatted and readable.
    *   Confirm that a `demo_analysis_report.txt` file is created with the analysis details.

2.  **Interactive Job Fit Analysis:**
    *   Run the interactive analysis (Option 2 in main menu).
    *   Provide sample resume and job description texts (ensure they meet the minimum length requirement).
    *   Verify that the analysis proceeds and generates similar detailed output as the demo mode.
    *   Check for appropriate error handling if input texts are too short or invalid.
    *   Confirm that an `interactive_analysis_report.txt` file is created.

3.  **API Key Configuration Error Handling:**
    *   (Optional: Temporarily remove or invalidate the `GOOGLE_API_KEY` from Colab secrets if testing this scenario.)
    *   Run any analysis function (demo or interactive).
    *   Verify that an `API Configuration Error` message is displayed, indicating the missing API key.

4.  **JSON Response Parsing Error Handling:**
    *   (Difficult to test directly without modifying the model's output, but ensure the `analyze_job_fit` function includes `json.JSONDecodeError` handling).
    *   If the AI model were to return non-JSON, ensure an error message related to invalid JSON is printed.

5.  **General Resume Improvements (Manual Call):**
    *   Execute the `generate_resume_improvements` function with a sample resume.
    *   Verify that the `display_resume_improvements` function presents a list of general suggestions for resume enhancement.
    *   Check for appropriate error handling if the resume text is too short for general improvements.

6.  **Input Validation:**
    *   When prompted for resume or job description in interactive mode, enter very short text.
    *   Verify that the system correctly prompts the user to enter more detailed text.

In [121]:
import json
import re

def generate_resume_improvements(resume_text: str) -> dict:
    """
    Gets general resume improvement suggestions from the AI.
    """
    # Ensure validate_input is accessible, potentially re-import or assume it's global
    if 'validate_input' not in globals():
        raise RuntimeError("validate_input function is not defined. Please ensure text preprocessing functions are loaded.")

    if not validate_input(resume_text, min_length=100): # Higher min_length for general advice
        raise ValueError("Resume text is too short or invalid for general improvement suggestions. Minimum 100 characters required.")

    # Define the desired JSON output format for improvements
    improvement_format = json.dumps({
      "strengths": ["<list of 3 strengths>"],
      "weaknesses": ["<list of 3 weaknesses>"],
      "formatting_tips": ["<list of 3 formatting improvements>"],
      "content_tips": ["<list of 3 content improvements>"],
      "action_verbs": ["<list of 5 strong action verbs to use>"],
      "overall_score": "<number 0-100>"
    }, indent=2)

    prompt = f"""
    You are an expert career coach. Analyze the following resume and provide comprehensive, actionable suggestions to improve it.
    These suggestions should not be specific to a particular job, but rather general best practices for resume enhancement.

    Here is the RESUME:
    {resume_text}

    Provide your suggestions in a JSON format strictly adhering to the following structure:
    {improvement_format}

    Return ONLY valid JSON.
    """

    try:
        if 'model' not in globals():
            raise RuntimeError("Gemini model not initialized. Please run API configuration step.")

        response = model.generate_content(prompt)
        ai_response_text = response.text

        # Remove markdown code block fences if present
        ai_response_text = ai_response_text.replace('```json', '').replace('```', '').strip()

        try:
            parsed_response = json.loads(ai_response_text)
            return parsed_response
        except json.JSONDecodeError as e:
            print(f"JSON parsing error for resume improvements: {e}")
            print(f"AI Raw Response: {ai_response_text}")
            raise ValueError("AI response for resume improvements is not valid JSON.")

    except Exception as e:
        print(f"API call error for resume improvements: {e}")
        raise RuntimeError(f"Failed to get AI resume improvements: {e}")


def display_resume_improvements(improvements: dict) -> None:
    """
    Formats and presents general resume improvement suggestions.
    """
    if "error" in improvements:
        print(f"\n❌ Error: {improvements['error']}")
        return

    print("\n" + "=" * 70)
    print("💡 RESUME IMPROVEMENT SUGGESTIONS")
    print("=" * 70)

    print(f"\n📊 Overall Resume Score: {improvements.get('overall_score', 'N/A')}/100")

    print(f"\n✅ STRENGTHS:")
    for strength in improvements.get('strengths', []):
        print(f"   • {strength}")

    print(f"\n⚠️ AREAS TO IMPROVE:")
    for weakness in improvements.get('weaknesses', []):
        print(f"   • {weakness}")

    print(f"\n🎨 FORMATTING TIPS:")
    for tip in improvements.get('formatting_tips', []):
        print(f"   • {tip}")

    print(f"\n✍️ CONTENT TIPS:")
    for tip in improvements.get('content_tips', []):
        print(f"   • {tip}")

    print(f"\n💪 STRONG ACTION VERBS TO USE:")
    for verb in improvements.get('action_verbs', []):
        print(f"   • {verb}")

    print("\n" + "=" * 70 + "\n")

print("Functions `generate_resume_improvements` and `display_resume_improvements` defined successfully.")

Functions `generate_resume_improvements` and `display_resume_improvements` defined successfully.


In [123]:
if __name__ == '__main__':
    main()


Welcome to the AI-powered Job Application Assistant!

--- Choose an Option ---
1. Run Demo Analysis
2. Run Interactive Analysis
3. Get General Resume Improvements
4. Exit
Enter your choice (1, 2, 3, or 4): 3

--- General Resume Improvements ---

Enter your RESUME text for general improvements (minimum 100 characters):
John Doe     john.doe@email.com | (123) 456-7890 | LinkedIn: linkedin.com/in/johndoe      Summary     Highly motivated software engineer with 5 years of experience in Python, Java, and cloud platforms (AWS, Azure). Proven ability to develop and deploy scalable web applications and data processing pipelines.      Skills     Programming: Python, Java, JavaScript, C++     Web Technologies: Django, Flask, React, Node.js     Databases: PostgreSQL, MongoDB, SQL     Cloud Platforms: AWS, Azure, Docker, Kubernetes     Tools: Git, Jira, Agile, Scrum     Data Science: Pandas, NumPy, Scikit-learn, Machine Learning      Experience     Senior Software Engineer | Tech Solutions Inc. |